In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_csv("data/stock_prices.csv")

In [0]:
def get_final_data(data):
  data = data.drop(columns=["Date", "Adj Close"])
  values = data.values
  return values[:, [0, 1, 2, 3, 4]], values[:, 3]

In [0]:
def get_final_y(y_data):
  y_data = list(y_data)
  y_data.pop(0)
  y_data.append(None)
  return y_data

In [0]:
def get_prepared_data(data):
  data, y_data= get_final_data(data)
  shifted_data = get_final_y(y_data)
  return data[:-1], shifted_data[:-1]

In [0]:
def train_test_split(X, Y, train_freq=0.8):
  lx = len(X)
  split_train = int(train_freq * lx)
  train_x, train_y = X[:split_train], Y[:split_train]
  test_x, test_y = X[split_train:], Y[split_train:]
  return train_x, train_y, test_x, test_y

In [0]:
def prep_data(train_x, train_y):
    train_x, train_y, test_x, test_y = train_test_split(train_x, train_y)
    train_x=np.array(np.expand_dims(train_x, axis=1))
    train_y=np.array(train_y)
    test_x=np.array(np.expand_dims(test_x, axis=1))
    test_y=np.array(test_y)

    # print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)
    return train_x, train_y, test_x, test_y

In [0]:
import tensorflow as tf
from keras import backend as K
import random as rn
rs=5
np.random.seed(rs)
rn.seed(rs)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
tf.set_random_seed(rs)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)


In [0]:
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM

In [0]:
def define_lstm_model(batches, input_shape):
  model = Sequential()
  model.add(LSTM(60, input_shape = input_shape, stateful=True, batch_input_shape=(batches, input_shape[0], input_shape[1])))
  model.add(Dense(1))
  return model

In [0]:
def train_model(train_x, train_y, test_x, test_y, epochs=110, batches=1):
  model = define_lstm_model(batches, (train_x.shape[1], train_x.shape[2]))
  model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse','mape'])
  model.fit(train_x, train_y, verbose=2, epochs=epochs, batch_size=batches, validation_data=(test_x, test_y), shuffle=False)
  return model

In [280]:
train_x, train_y=get_prepared_data(data)
train_x,train_y,test_x,test_y=prep_data(train_x, train_y)
model=train_model(train_x, train_y, test_x, test_y)

Train on 200 samples, validate on 50 samples
Epoch 1/110
 - 3s - loss: 104328.2097 - mean_squared_error: 104328.2097 - mean_absolute_percentage_error: 99.6421 - val_loss: 103683.8992 - val_mean_squared_error: 103683.8992 - val_mean_absolute_percentage_error: 99.0881
Epoch 2/110
 - 0s - loss: 101901.2967 - mean_squared_error: 101901.2967 - mean_absolute_percentage_error: 98.4591 - val_loss: 101249.4827 - val_mean_squared_error: 101249.4827 - val_mean_absolute_percentage_error: 97.9088
Epoch 3/110
 - 0s - loss: 99494.2432 - mean_squared_error: 99494.2432 - mean_absolute_percentage_error: 97.2725 - val_loss: 98850.7266 - val_mean_squared_error: 98850.7266 - val_mean_absolute_percentage_error: 96.7327
Epoch 4/110
 - 1s - loss: 97124.5354 - mean_squared_error: 97124.5354 - mean_absolute_percentage_error: 96.0902 - val_loss: 96490.2489 - val_mean_squared_error: 96490.2489 - val_mean_absolute_percentage_error: 95.5613
Epoch 5/110
 - 0s - loss: 94793.1879 - mean_squared_error: 94793.1879 - mea

In [0]:
train_score = model.evaluate(train_x, train_y, verbose=0, batch_size=1)

In [0]:
test_score = model.evaluate(test_x, test_y, verbose=0, batch_size=1)

In [284]:
train_score, test_score

([703.1921998626366, 703.1921998626366, 6.950832686051727],
 [431.1745428594947, 431.1745428594947, 4.879801982492208])

In [0]:
to_predict=[[296.94,298.19,288.00,288.95,8350500], 280.74]
data_shape=(1,5)

In [286]:
p=np.array(to_predict[0])
p=np.reshape(p, (1, data_shape[0], data_shape[1]))
x=model.predict(p, batch_size=1)
x

array([[319.73438]], dtype=float32)